<h5 style="font-weight:bold;">Name: Ou Wang</h5>
<h5 style="font-weight:bold;">uID: u1275903</h5>
<h2 style="font-weight:bold;text-align:center;">Are the daily supplied of finished motor gasoline and the confirmed cases of covid-19 correlated in the US?</h2>
<p>CDC has been recommendingsocial distancingand quarantine,and most of theorganizations/companieshave been keeping their employeesworking from home.The intuitionis that the daily supplied gasoline may decrease during the covid-19 pandemicin US. Theline plot in Figure 1Aindeedshowedthat the gasoline supply started decreasing since 03/13/2020, when many companies startedtheworkingfrom home policies(from > 8,000thousand barrels per daybefore 03/13/2020to 5,000thousand barrels per dayon04/20/2020).</p>

<p>The barplot in Figure 1Aindicated thatthe daily confirmed cases of covid-19 started showing a trend of decreasingbetween 04/10/2020and 06/05/2020, which isonemonth afterthe gasoline supply starteddropping. However, the Pearsoncorrelation coefficient of -0.33 (Figure 1A)did not suggesta strong correlationbetweengasoline supply and covid-19 cases.This could be due to thatthe changesof covid-19 cases and gasoline supply are not simultaneous.Also, the gas supply graduallyincreased back since 04/03/2020, and it reached to 8,000thousand barrels perday again on 06/19/2020.Thus, the data points could be insufficient to show the correlation between these two variables.</p>

<p>Cumulative sum analysiswas then performedto detect changes for gasoline supply and covid-19 cases(Figure 1B and 1C).The resultsindicatedthat the gasolinesupply decreasedbetween 03/06/2020 and 04/24/2020, and the confirmedcases of covid-19decreased between 04/24/2020 and 06/19/2020, suggestingthat reduced gasolinesupply may be associated withthereduced casesofcovid-19. However,and itmaytakeabout a month to showthe effect. Thereason could be that reduced gasoline supplywould lead to reduced human traveling, which decreasesthe chanceof human contactand spreadingcovid-19.</p>

<p>In summary, although finished motor gasolinesupply and confirmedcasesof covid-19 did not showstrong correlation, the cumulativesum analysis suggested that the daily cases of covid-19 indeed decreased,following the decease of motor gasoline supply. In addition,as shown in Figure 1A, sincethe motor gasolinesupplyincreased back to > 8,000thousand barrels per dayon06/19/2020, the daily cases of covid-19 started skyrocketing.Therefore, weshould follow CDC’s recommendationto stay at hometo helpflattenthe curve. </p>

In [10]:
# !export GOOGLE_APPLICATION_CREDENTIALS="/home/owang/.google/BigQuery.json"
%load_ext google.cloud.bigquery

The google.cloud.bigquery extension is already loaded. To reload it, use:
  %reload_ext google.cloud.bigquery


In [11]:
%%bigquery df_cases_gas --project bigquery-281204 --use_rest_api
SELECT date
    , daily_confirmed_cases
    , gas 
FROM 
    `bigquery-281204.is6850.summary_daily_cases`,
    unnest(`bigquery-281204.is6850.summary_daily_cases`.summary)
;

<h2 style="color:blue;">create dim_date table </h2>

In [ ]:
%%bigquery --project bigquery-281204 --use_rest_api
create or replace table bigquery-281204.is6850.dim_date
    as (
        select 
            cast(format_date("%E4Y%m%d", dates) as numeric) as `DateKey`
            , format_date("%D", dates) as `Date`
            , extract(year from dates) as `Year`
            , extract(month from dates) as `Month`
            , extract(quarter from dates) as `Quarter`
            , extract(dayofweek from dates) as `DayOfWeek`
            , extract(isoweek from dates) as `ISOWeek`
            , format_date("%A", dates) as `DayName`
            , format_date("%B", dates) as `MonthName`
            , if(extract(dayofweek from dates)=1 or extract(dayofweek from dates)=7, 1, 0) as `IsWeekend`
        from unnest(generate_date_array('2010-01-01', '2030-01-01')) as `dates`
);

In [3]:
%%bigquery --project bigquery-281204 --use_rest_api
select *
from is6850.dim_date
limit 5
;

,DateKey,Date,Year,Month,Quarter,DayOfWeek,ISOWeek,DayName,MonthName,IsWeekend
0,20100224,02/24/10,2010,2,1,4,8,Wednesday,February,0
1,20100727,07/27/10,2010,7,3,3,30,Tuesday,July,0
2,20100821,08/21/10,2010,8,3,7,33,Saturday,August,1
3,20100830,08/30/10,2010,8,3,2,35,Monday,August,0
4,20100914,09/14/10,2010,9,3,3,37,Tuesday,September,0


<h2 style="color:blue;"> create covid-19 summary table, including daily cases/death and accumulative cases/death </h2>

In [ ]:
%%bigquery --project bigquery-281204 --use_rest_api
create or replace table bigquery-281204.is6850.covid19_summary as
    with temp as (
      select date
        , sum(confirmed_cases) as cumulative_confirmed_cases
        , sum(deaths) as cumulative_deaths
      from `bigquery-public-data.covid19_nyt.us_states`
      group by date
      order by date
    )
    select date
     , cumulative_confirmed_cases
     , cumulative_confirmed_cases - first_value(cumulative_confirmed_cases) over (diff) as daily_confirmed_cases
     , cumulative_deaths
     , cumulative_deaths - first_value(cumulative_deaths) over (diff) as daily_death
    from temp
    window diff as (
      order by date
      rows between 1 preceding and current row
    );

In [4]:
%%bigquery --project bigquery-281204 --use_rest_api
select *
from is6850.covid19_summary
order by date
limit 5
;

,date,cumulative_confirmed_cases,daily_confirmed_cases,cumulative_deaths,daily_death
0,2020-01-21,1,0,0,0
1,2020-01-22,1,0,0,0
2,2020-01-23,1,0,0,0
3,2020-01-24,2,1,0,0
4,2020-01-25,3,1,0,0


<h2 style="color:blue;">create summary table include gas consumption and daily covid-19 cases, saved in array to preserve order </h2>

In [ ]:
%%bigquery --project bigquery-281204 --use_rest_api
create or replace table bigquery-281204.is6850.summary_daily_cases as
  with temp as (
    select 
        date
        , daily_confirmed_cases
        , Weekly_US_Product_Supplied_of_Finished_Motor_Gasoline_Thousand_Barrels_per_Day as gas
    from bigquery-281204.is6850.covid19_summary dc
    join bigquery-281204.is6850.gas_consumption gc
    on dc.date = gc.week_of
  )
  select 
    array_agg(
      struct(date, daily_confirmed_cases, gas)
      order by date) 
      as summary
  from temp
;

In [5]:
%%bigquery --project bigquery-281204 --use_rest_api
SELECT date
    , daily_confirmed_cases
    , gas 
FROM 
    `bigquery-281204.is6850.summary_daily_cases`,
    unnest(`bigquery-281204.is6850.summary_daily_cases`.summary)
limit 5
;

,date,daily_confirmed_cases,gas
0,2020-01-24,1,8793
1,2020-01-31,1,8933
2,2020-02-07,0,8722
3,2020-02-14,0,8918
4,2020-02-21,3,9035


<h2 style="color:blue;">get standard devidation and mean then normalize data </h2>

In [ ]:
%%bigquery --project bigquery-281204 --use_rest_api
create or replace table bigquery-281204.is6850.correlation_data as
with temp as (
  SELECT 
    date
    , daily_confirmed_cases
    , stddev(daily_confirmed_cases)
      over(statistics) as cases_stddev
    , avg(daily_confirmed_cases) 
      over(statistics) as cases_avg
    , gas
    , stddev(gas)
      over(statistics) as gas_stddev
    , avg(gas)
      over(statistics) as gas_avg
  FROM `bigquery-281204.is6850.summary_daily_cases`
  , unnest(`bigquery-281204.is6850.summary_daily_cases`.summary)
  window statistics as (
      order by date
      rows between unbounded preceding and unbounded following
  )
)
select
  date,
  (daily_confirmed_cases - cases_avg)/cases_stddev as norm_cases
  , (gas - gas_avg)/gas_stddev as norm_gas
from temp
;

<h2 style="color:blue;">get Pearson Correlation coefficientcreate from above normalized data</h2>

In [6]:
%%bigquery --project bigquery-281204 --use_rest_api
SELECT round(corr(norm_cases, norm_gas), 4) as pearson
FROM `bigquery-281204.is6850.correlation_data`
;

,pearson
0,-0.3252


<h2 style="color:blue;">Prepare data for cumulative sum calculation</h2>

In [18]:
%%bigquery --project bigquery-281204 --use_rest_api
create or replace table is6850.cusum_data as
with temp as (
SELECT 
  date
  , daily_confirmed_cases
  , avg(daily_confirmed_cases)
    over(moving_avg) as cases_c
  , stddev(daily_confirmed_cases)
    over(std_window) as case_t
  , gas
  , avg(gas)
    over(moving_avg) as gas_c
  , stddev(gas)
    over(std_window) as gas_t
FROM `bigquery-281204.is6850.summary_daily_cases`
, unnest(`bigquery-281204.is6850.summary_daily_cases`.summary)
window moving_avg as (
    order by date
    rows between 3 preceding and current row
    ), 
    std_window as (
      order by date
      rows between 3 preceding and current row
      )
)
select 
  * replace(
    cases_c*1.1 as cases_c
    , gas_c*1.1 as gas_c
    , case_t*2 as case_t
    , gas_t*2 as gas_t
    )
from temp
where date > "2020-03-01"
;

""


In [7]:
%%bigquery --project bigquery-281204 --use_rest_api
select *
from is6850.cusum_data
limit 5
;

,date,daily_confirmed_cases,cases_c,case_t,gas,gas_c,gas_t
0,2020-06-26,45528,35904.550,17735.875545,8561,9058.225,808.995056
1,2020-04-03,32318,15351.325,28227.940975,5065,8320.675,4199.996627
2,2020-06-12,25492,28199.050,4324.318983,7870,8407.300,609.386577
3,2020-06-19,30835,30135.050,5805.946492,8608,8779.925,893.407522
4,2020-03-27,17330,6486.700,16046.584268,6659,9526.275,2764.331565


<h2 style="color:blue;">Calculate cumulative sum</2>

In [21]:
%%bigquery --project bigquery-281204 --use_rest_api
create or replace table is6850.cusum_table as
with temp as(
  SELECT *
  FROM `bigquery-281204.is6850.cusum_data`
  order by date
), 
temp2 as(
  select 
    date
    , case_t, 
    daily_confirmed_cases - cases_c as sum_cases, 
    gas_t, gas - gas_c as sum_gas
  from temp
)
select *
  , if(sum(sum_cases) over(cusum_win) > 0, 0, sum(sum_cases) over(cusum_win)) as cusum_case
  , if(sum(sum_gas) over(cusum_win) > 0, 0, sum(sum_gas) over(cusum_win)) as cusum_gas
from temp2
where date > "2020-03-01"
window cusum_win as (
  order by date
  rows between 1 preceding and current row
)
;

""


In [8]:
%%bigquery --project bigquery-281204 --use_rest_api
select * 
from is6850.cusum_table
limit 5
;

,date,case_t,sum_cases,gas_t,sum_gas,cusum_case,cusum_gas
0,2020-04-03,28227.940975,16966.675,4199.996627,-3255.675,0.000,-6122.950
1,2020-06-12,4324.318983,-2707.050,609.386577,-537.300,-2403.975,-747.600
2,2020-05-29,3455.894674,-3564.700,656.139213,-423.250,-10473.450,-899.125
3,2020-05-01,4316.360929,-3345.575,1407.605058,362.100,-3465.850,0.000
4,2020-03-20,5426.222596,3896.675,735.867742,-1384.200,0.000,-1963.850
